In [1]:
import pandas as pd

import re
import itertools
from collections import Counter
from typing import  Dict, List, Tuple

from tqdm import tqdm
from transformers import AutoTokenizer

2021-09-30 16:10:19.985745: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
Train_csv_file = '/opt/ml/dataset/train/train.csv'
Test_csv_file = '/opt/ml/dataset/test/test_data.csv'

df = pd.read_csv(Train_csv_file)
df = df.drop_duplicates(['sentence','subject_entity','object_entity','label'])


MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

vocab = list(tokenizer.get_vocab().keys())
unused_list = [word for word in vocab if word.startswith('[unused')]        
print(f'unused token count: {len(unused_list)}')

/opt/conda/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


unused token count: 500


In [3]:
# # UNK_token_string
# unk_idx = tokenizer.get_vocab()[tokenizer.unk_token]
# UNK_word_list = [word for sen in tqdm(df['sentence']) for word in sen.split() if tokenizer.encode(word)]
# UNK_word_counter = Counter(UNK_word_list)

# 🛰️ UNK 분포

In [6]:
def UNK_word(df_col:pd.DataFrame, unk_idx:int) -> List[Tuple] :
    UNK_word_idx = []
    for i, word in enumerate(tqdm(df_col)) :
        if unk_idx in tokenizer.encode(word) :
            UNK_word_idx.append((i,word))
    return UNK_word_idx

In [7]:
unk_idx = tokenizer.get_vocab()[tokenizer.unk_token] # 3

sub_col = df['subject_entity'].apply(eval).str['word']
sub_UNK = UNK_word(sub_col, unk_idx)

obj_col = df['object_entity'].apply(eval).str['word']
obj_UNK = UNK_word(obj_col, unk_idx)

sen_col = df['sentence']
sen_UNK = UNK_word(sen_col, unk_idx)

100%|██████████| 32428/32428 [00:07<00:00, 4142.04it/s]


In [8]:
print(len(sub_UNK), len(obj_UNK), len(sen_UNK))
print(sub_UNK[0]) # (df row index, entity)
print(obj_UNK[0]) # (df row index, entity)
print(sen_UNK[0]) # (df row index, senetence)

223 282 3217
(21, '스티브 바라캇')
(191, '少弐氏')
(7, '박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.')


## 🛸 한 단어로 이루어지지 않은 entity
- 여러 단어
- 특정 패턴이 존재하는
- 맞추기 어려울 것 같은
+ 위 내용을 만족하는 entity는 entity token wrap 해주는게 어떠려나?

In [9]:
for obj in df['object_entity'] :
    d_obj = eval(obj)
    if d_obj['type'] == 'DAT' :
        if len(d_obj['word'].split()) > 1 :
            print(obj)
            test_encoced = tokenizer.encode(d_obj['word'])
            print(test_encoced)
            Id_to_tokens = tokenizer.convert_ids_to_tokens(test_encoced[1:-1])
            print(Id_to_tokens)
            break

{'word': '1937년 4월 29일', 'start_idx': 9, 'end_idx': 20, 'type': 'DAT'}
[0, 20533, 2440, 24, 2429, 4346, 2210, 2]
['1937', '##년', '4', '##월', '29', '##일']


# 🛰️UNK으로 변하는 word 및 char 확인

In [10]:
test_encoced = tokenizer.encode(obj_UNK[13][1])

print(test_encoced)
Id_to_tokens = tokenizer.convert_ids_to_tokens(test_encoced[1:-1])
Known_char = []
for word in Id_to_tokens :
    if word != tokenizer.unk_token :
        string = word.replace('#','')
        Known_char.append(string)

[0, 20482, 3, 2]


In [11]:
def Convert_tokenizer_text(text:str) -> List[str]: ## text => tokenizer ids => tokens
    ID_encoced = tokenizer.encode(text) ## text to ids
    ID_to_tokens = tokenizer.convert_ids_to_tokens(ID_encoced[1:-1]) ## ids to token
    return ID_to_tokens

def subword_comparison(wordpiece:List) -> List[str]: ## subword # 제거용
    Known_char = []
    for subword in wordpiece :
        if subword == tokenizer.unk_token :
            Known_char.append(tokenizer.unk_token)
        else :
            string = subword.replace('#','')
            Known_char.extend(string)
    return Known_char


def UNK_word_and_chr(text:str) -> Tuple[List[str], List[str]]:
    word_UNK_list = []
    sub_word_UNK_list = []
    
    wordpieces_tokens = Convert_tokenizer_text(text)

    words_list = text.split()
    tokens_list = tokenizer.convert_tokens_to_string(wordpieces_tokens).split()
    for word, token in zip(words_list, tokens_list) :
        if word != token :
            UNK_ID_encoded = tokenizer.encode(token)[1:-1]
            if UNK_ID_encoded == [unk_idx] :
                word_UNK_list.append(word)
            else :
                subwordpieces_ID_encoded = Convert_tokenizer_text(word)
                Known_subword = subword_comparison(subwordpieces_ID_encoded)
                for sub_char, NK_char in zip(word, Known_subword) :
                    if sub_char != NK_char :
                        sub_word_UNK_list.append(sub_char)
    return word_UNK_list, sub_word_UNK_list

In [12]:
from IPython.core.display import HTML
def word_highligt_html(txt, word, color='black', highlight=None, attr=None):
    if isinstance(word, str):
        txt = txt.replace(word, f'<span style="color: {color}; background-color:{highlight}">{word}</span>')
    else:
        if not isinstance(color, list):
            color = [color] * len(word)
        if not isinstance(highlight, list):
            highlight = [highlight] * len(word)
        for w, c, h in zip(word, color, highlight):
            txt = txt.replace(w, f'<span style="color: {c}; background-color:{h}">{w}</span>')
    return txt

In [21]:
len(tokenizer.get_vocab().keys()), tokenizer.get_vocab().keys()

(32000,
 dict_keys(['천주교', '사바', '뚫어지', '스킨', '##뻥', '마모', '얽힌', '박영', '생육', '판다', '오프', '피아노', '빙', '굵직', '##BA', '거늘', '서산', '증명', '반란', '초유', '퇴원', 'Americ', '集', '일환', 'KAI', '저지르', '백합', '담뱃값', '회한', '입장권', '직행', '이번', '자이언츠', '으흠', '마네', '[unused185]', '요미우리', '도요타', '완공', 'pr', '끌어올렸', '찾아낸', '악장', '##곡', '위스', '버려야', '키울', '탄소', '금할', '서상', '배워야', '대학교', '이강', '##어난', '밀가루', '앞선', '이청', 'stud', '배설', '침체', '의외', '246', '야대', '남부', '주입', '롯데제과', '에요', '의왕', '실상', '외', '날렸', '백정', '장악', '##af', '##린다는', '흩어지', '개그콘서트', '##uch', '##자치', '계획', '떨어져서', '62', '범', '갈랐', '제대로', '어떻든', '파이널', '##다닌다', '이곳', '더해져', '누런', '속삭이', '강원도', '펼칠', '뒷전', '박테리아', '오른다', '노면', '마담', '모닥불', '사단', '배제', '다행히', '판매량', '매달', '##es', 'CSR', '카누', '형주', '귀요미', '축구팀', '총대', '볶음밥', '감탄사', '##evelop', '겔', '스키장', '##자매', '[unused239]', '어지', '미룬', '쓴', '뜻밖', '골문', '멘', '오리엔', '숙박', '프로필', '수아', '살벌', '자비', 'day', '시화', '##팽이', '다원', '안내문', '패인', '론스타', '##믿', '父', '23', '이채', '전생', '케이트', '자면', '남아프리카', '

In [32]:
txt = ''
count = 1
unk_len = 0
for idx, sen in tqdm(sen_UNK) :
    UNK_word, UNK_subword = UNK_word_and_chr(sen)
    merged_UNK = UNK_word + UNK_subword
    unk_len += len(merged_UNK)
    txt += word_highligt_html(sen, merged_UNK, ['white']*len(merged_UNK),  ['#96C4ED']*len(UNK_word)+['#B19CD9']*len(UNK_subword)) + '<br/><br/>'
    if count > 100:
        break
    count += 1
HTML(txt)

  3%|▎         | 100/3217 [00:00<00:08, 360.27it/s]


In [30]:
not_hangul = []
for idx, sen in sub_UNK+obj_UNK :
    UNK_word, UNK_subword = UNK_word_and_chr(sen)
    not_hangul += UNK_word
    not_hangul += UNK_subword
print(len(not_hangul), not_hangul)

593 ['바라캇', '슌', '리셴녠', '사르키샨', '흄', '켐페스', '쟝쩐위', '넋업샨', '킵루츠', '리콴유', '푈러', '뮐러', '홋카이도', '피녜라', 'MŠK', '훙윈', '로뮈알', '넵투누스', '푸르트벵글러', '비욘세', '리옌훙', '베렝가리오', '에스파뇰', '루쳄부르스키', '꽈드로스', '래이쾨넨', '로퀜스', '뱌체슬라프', '훙윈', '리셴녠', '쥘', '숀', '포로셴코', '쥘', '스웜', '헹크', '칸쟈니∞', '긱스', '스뱌토폴크', '베렝가리오', '메이헴', '쥰', '욘', '슌', '방얏', '핌', '카뮈', '에스파뇰', '쾰른', '뢴트겐', '베렝가리오', '넋업샨', '묀헨글라트바흐', '숀', '헴스워스', '아뎀', '슌', '홋스퍼', '쟝쩐위', '체르노젬스키', '로코솝스키', '포로셴코', '숀', '숀', '귈', '슈쳉스니', '옝제이치크', '쥘', '푀', '쥘', '홋스퍼', '홋스퍼', '엡손', '쥘', '힉스', '옝제이차크', '다롄', '뮈르달', '뮐러', '숀', '베렝가리오', '홋스퍼', '다롄', '우젠슝', '쿠틉', '쟝쩐위', '홋스퍼', '하이츨러', '위키트리뷴', '뵐플리', '야쿱', '킵', '젭', '아믈리브', '하이츨러', '리콴유', '로퀜스', '우젠슝', '묀헨글라트바흐', '홋스퍼', '로바쳅스키', '포로셴코', '쥘', '호엔촐레른지크마링겐', '헹', '우젠슝', '카녜이', '홋스퍼', '베렝가리오', '홋스퍼', '래이쾨넨', '웸블리', '다롄', '욘', '슈쳉스니', '昶', '위쾨넨', '뮐러', '쑹타오', '횃불트리니티신학대학원대학교', '홋스퍼', '홋카이도', '퓌르트', '펍메드', '파스콸', '에스파뇰', '슌', '리셴녠', '카뮈', '우젠슝', '맬', '흄', '젬베콜라', '포로셴코', '퓌르트', '훙윈', '아르툠', 'RÚV', '홋스퍼', '홋카이도', '뮈르달', '퍼퓸'

In [17]:
for idx, sen in sen_UNK :
    if re.search(r'[äöüÄÖÜß]',sen):
        UNK_word, UNK_subword = UNK_word_and_chr(sen)
        print(UNK_word, UNK_subword, sen)

['푈러', '에'] ['V', 'ö'] 루디 푈러 (Rudi Völler) 에 의해 2002년 FIFA 월드컵 독일 대표팀에도 발탁되었으며, 1998년부터 2002년까지 독일 대표팀에서 33경기에 나서 10골을 넣었다.
['소련과'] ['H'] 핀란드 방위군의 저격수인 시모 해위해(Simo Häyhä 1925-2002)는 소련과 핀란드의 분쟁인 겨울전쟁에서 무려 542명의 사살기록을 세웠다.
['뮐러(Hermann'] ['M', 'ü'] 헤르만 뮐러(Hermann Müller, 1879년 5월 18일 ~ 1931년 3월 20일)는 바이마르 공화국 시기에 주로 활동한 독일의 정치인이다.
['입단한'] ['T', 'Ö'] 1994년 스웨덴의 여자 축구 클럽인 튀레쇠 FF(Tyresö FF)에 입단한 이래 미국의 워싱턴 워트호그스(Washington Warthogs, 1995년)와 델라웨어 지니스(Delaware Genies, 1998년), 보스턴 브레이커스(Boston Breakers, 2001년~2003년), 스웨덴의 KIF 외레브로(KIF Örebro DFF, 2005년) 소속으로 뛰었고 2009년 미국 여자 프로 축구 소속 클럽인 보스턴 브레이커스로 복귀했다.
['을'] ['F', 'u', 'ß', 'b', 'a', 'l', 'M', 'ü'] MTV 1879의 선수들이 1900년 2월 27일 집회에서, 축구부가 독일 축구 협회 (DFB) 에 가맹이 불가능함에 따라, 11명의 축구부 선수들이 MTV 1879에서 탈퇴하여 같은날 오후에 축구 클럽 바이에른 뮌헨 (Fußball-Club Bayern München) 을 창단하였다.
[] ['S', 'c'] 페르디난트 쇠르너(Ferdinand Schörner, 1892년 6월 12일 ~ 1973년 7월 2일)은 제1차 세계대전, 제2차 세계대전에서 활약한 독일 군인이다.
['에드문트'] ['P', 'h', 'ä'] 현상학(Phänomenologie)의 창시자 에드문트 후설은 독일에서 태어나 할레 대학교의 강사, 괴

In [18]:
for idx, sen in sub_UNK+obj_UNK :
    if re.search(r'[äöüÄÖÜß]',sen):
        UNK_word, UNK_subword = UNK_word_and_chr(sen)
        print(UNK_word, UNK_subword)

['Müller'] []
['Hoeneß'] []
['Päffgen'] []
['Wölfli'] []
['Südtirol'] []
['ÖBB'] []
['Jürgen'] []
['Atatürk'] []


# 🛰️시험

In [19]:
Test_csv_file = '/opt/ml/dataset/test/test_data.csv'
test_df = pd.read_csv(Test_csv_file)

In [20]:
t_sub_col = test_df['subject_entity'].apply(eval).str['word']
t_sub_UNK = UNK_word(t_sub_col, unk_idx)

t_obj_col = test_df['object_entity'].apply(eval).str['word']
t_obj_UNK = UNK_word(t_obj_col, unk_idx)

t_sen_col = test_df['sentence']
t_sen_UNK = UNK_word(t_sen_col, unk_idx)

TypeError: 'list' object is not callable

In [ ]:
for idx, sen in t_sub_UNK+t_obj_UNK :
    if re.search(r'[äöüÄÖÜß]',sen):
        UNK_word, UNK_subword = UNK_word_and_chr(sen)
        print(UNK_word, UNK_subword)

In [ ]:
for idx, sen in t_sen_UNK :
    if re.search(r'[äöüÄÖÜß]',sen):
        UNK_word, UNK_subword = UNK_word_and_chr(sen)
        print(UNK_word, UNK_subword, sen)

['Wiesehöfer는'] [] 독일 역사 학자 Josef Wiesehöfer는 키루스를 인권 부문 선구자로 묘사 한 것은 이란의 이미지메이킹 전략이라고 언급했다.
['Lübeck(뤼벡)'] ['–', 'w', 'e', 'i', 'ß', '-', 'r', 'o', 't', '(', '흑', '백', '적', '-', '나', '치'] 1985년 Lübeck(뤼벡) 콘서트에는 약700명의 오이! 스킨헤드들과 소수의 네오나치들이 운집하였는데, 케빈은 Deutschlandlied의 한 구절을 'schwarz-rot-gold(현재독일 깃발 색)' 대신 'schwarz–weiß-rot(흑백적-나치독일의 깃발)' 이라고 바꾸어 불렀다.
['窩闊台汗國)은', '3남인', '오고타이가', '칭기스', '칸에게서'] ['Ö'] 오고타이국, 또는 오고타이 칸국(Ögödei Khan國, 窩闊台汗國)은 칭기스 칸의 3남인 오고타이가 칭기스 칸에게서 영지를 받아 성립된 몽골제국의 울루스이다.
['자유'] ['G', 'r', 'o', 'ß', '-', 'H', 'a', 'm', 'b', 'u', 'r', 'g', '-', 'G', 'e', 's', 'e', 't', 'z'] 1937년 나치는 소위 대(大) 함부르크 법(Groß-Hamburg-Gesetz)이라 불리는 법을 제정해서 함부르크 자유 한자 시를 팽창시켜 프로이센 자유주의 슐레스비히홀슈타인 지방에 속했던 여러 마을까지 포괄하도록 하였다.
['가문인'] ['Ö', 's', 't'] 이 사건으로 인해 망누스는 1132년경에 스웨덴의 왕위에서 물러났고 외스테르예틀란드(Östergötland)의 대지주 가문인 스베르케르가(Sverker) 출신인 스베르케르 1세가 스웨덴의 국왕으로 즉위하게 된다.


In [ ]:
for idx, sen in t_sen_UNK :
    UNK_word, UNK_subword = UNK_word_and_chr(sen)
    
    print(UNK_word, UNK_subword, sen)

['대한민국의'] ['姜'] 강명구(姜明求, 1917년~2000년)는 대한민국의 건축가이다.
[] ['栗', '亭', '李', '寬'] 일찍이 이천(利川)의 율정(栗亭) 이관의(李寬義)의 문하에서 수학하였다.
['코냑'] [] 잉글랜드와 로마 사이의 친밀감은 1526년 코냑 동맹의 공식화에서 볼 수 있다.
['게재되어'] ['。'] 초기에는 대부분의 경우 '탈퇴'라는 표현을 사용하였지만(결성 당시부터 나카자와 졸업까지의 모닝구무스메。의 역사가 게재되어 있는 ASAYAN 공식 사이트 등에서는 지금도 탈퇴라는 표현으로 통일되어 있다.) 2001년 이후에는 공식적으로 주로 졸업이라는 표현을 쓰게 되어('졸업 메모리얼' 굿즈 등), 과거 멤버에 대해서도 졸업이라는 표현으로 통일한 경우도 많다(콘서트 MC에서의 '졸업 멤버' 호칭 등).
['다시', '현록대부(顯祿大夫)에'] ['興', '祿'] 1775년(영조 51) 4월 24일 영조의 특명으로 흥록대부(興祿大夫)에 가자되고, 이해 12월 10일 다시 현록대부(顯祿大夫)에 가자되었다.
['윰댕은'] [] 7일 윰댕은 자신의 유튜브 채널을 통해 대도서관과 만나기 전 결혼했었던 사실과 아이가 하나 있다고 밝혔다.
['스웁(Swoop)'] [] 지역 주민들의 글로벌 문화발전을 위해 마련된 이번 행사 1부에는 전 MBC 스포츠 해설위원이자 현재 NFL 코리아 회장을 지내고 있는 강윤종 전문해설위원이 미식축구에 익숙하지 못한 주민들을 대상으로 경기 설명을 진행하였으며, 주민들은 1부 행사 동안 마술쇼, 바비큐 파티, 스웁(Swoop) 유타대 마스코트 사진 촬영 등의 재미있는 활동을 진행하였으며, 2부에서는 강윤종, 주장훈 전문해설위원과 함께 라이브 해설을 포함한 단체 TV 중계 관전 행사를 진행하였다.
['모아', '한다.'] ['客', '客'] 춘신군은 아래에 식객(食客) 3천 인을 모아 거느리고 있었는데, 상객(上客)은 모두 구슬로 장식한 신발을 신고 있었다고 한다.
['비판해'] ['蘇'] 표면상 이유는 건강 문제였지만,